# Scopes explained
From the Docs:  

A namespace is a mapping from names to objects.
Examples of namespaces are: the set of built-in names (containing functions such as abs(), and built-in exception names); the global names in a module; and the local names in a function invocation. In a sense the set of attributes of an object also form a namespace. The important thing to know about namespaces is that there is absolutely no relation between names in different namespaces; for instance, two different modules may both define a function maximize without confusion — users of the modules must prefix it with the module name.
By the way, I use the word attribute for any name following a dot — for example, in the expression z.real, real is an attribute of the object z. Strictly speaking, references to names in modules are attribute references: in the expression modname.funcname, modname is a module object and funcname is an attribute of it. In this case there happens to be a straightforward mapping between the module’s attributes and the global names defined in the module: they share the same namespace!
***Namespaces are created at different moments and have different lifetimes.*** The namespace containing the built-in names is created when the Python interpreter starts up, and is never deleted. The global namespace for a module is created when the module definition is read in; normally, module namespaces also last until the interpreter quits. The statements executed by the top-level invocation of the interpreter, either read from a script file or interactively, are considered part of a module called __main__, so they have their own global namespace. (The built-in names actually also live in a module; this is called builtins.)

***The local namespace for a function is created when the function is called, and deleted when the function returns or raises an exception that is not handled within the function.*** (Actually, forgetting would be a better way to describe what actually happens.) Of course, recursive invocations each have their own local namespace.

A scope is a textual region of a Python program where a namespace is directly accessible. “Directly accessible” here means that an unqualified reference to a name attempts to find the name in the namespace.

Although scopes are determined statically, they are used dynamically. At any time during execution, there are 3 or 4 nested scopes whose namespaces are directly accessible:

the ***innermost scope, which is searched first, contains the local names***

the ***scopes of any enclosing functions, which are searched starting with the nearest enclosing scope, contains non-local, but also non-global names***

the ***next-to-last scope contains the current module’s global names***

the ***outermost scope (searched last) is the namespace containing built-in names***

***If a name is declared global, then all references and assignments go directly to the middle scope containing the module’s global names. To rebind variables found outside of the innermost scope, the nonlocal statement can be used; if not declared nonlocal, those variables are read-only (an attempt to write to such a variable will simply create a new local variable in the innermost scope, leaving the identically named outer variable unchanged).***

***Usually, the local scope references the local names of the (textually) current function. Outside functions, the local scope references the same namespace as the global scope: the module’s namespace.*** Class definitions place yet another namespace in the local scope.

***It is important to realize that scopes are determined textually: the global scope of a function defined in a module is that module’s namespace, no matter from where or by what alias the function is called. On the other hand, the actual search for names is done dynamically, at run time*** — however, the language definition is evolving towards static name resolution, at “compile” time, so don’t rely on dynamic name resolution! (In fact, local variables are already determined statically.)

***A special quirk of Python is that – if no global or nonlocal statement is in effect – assignments to names always go into the innermost scope. Assignments do not copy data — they just bind names to objects. The same is true for deletions: the statement del x removes the binding of x from the namespace referenced by the local scope. In fact, all operations that introduce new names use the local scope: in particular, import statements and function definitions bind the module or function name in the local scope.***

The global statement can be used to indicate that particular variables live in the global scope and should be rebound there; the nonlocal statement indicates that particular variables live in an enclosing scope and should be rebound there.

Let's look at an actual example!

In [11]:
def scope_test():
    def do_local():
        # this just creates a local variable in the do_local() namespaces
        spam = "local spam"

    def do_nonlocal():
        # this successfully assings the value to the scope_test() namespaces
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        # this successfully assings the value to the module namespace
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


To understand this code block better the following statements is very important:
> It is important to realize that scopes are determined textually: the global scope of a function defined in a module is that module’s namespace, no matter from where or by what alias the function is called. On the other hand, the actual search for names is done dynamically, at run time

So the search for names is done at run time, so where we try to access a value of a variable does play a role. For example if we were to print a value inside a function, then the interpreter would look inside that local scope of that function for that variable first and then inside the scope of the function that lies outside of the initial function. It would continue all the way to the module scope, if the variable still was not found. If the variable was still not found in the modul scope then the last resort would be the builtins namespace. If the variable is still not found then our familiar NameError will be thrown.  

The nested function tries to illustrate that point:

In [12]:
def illustrate_dynamic_search():
    def second_nested():
        # the second_nested function
        def first_nested():
            # the first_nested function
            def innermost():
                # the innermost function
                mail = "innermost mail"
                print("mail inside innermost():", mail)
            innermost()
            mail = "first_nested mail"
            print("mail inside first_nested():", mail)
        first_nested()
        mail = "second_nested mail"
        print("mail inside second_nested():", mail)
    second_nested()
        
illustrate_dynamic_search()

mail inside innermost(): innermost mail
mail inside first_nested(): first_nested mail
mail inside second_nested(): second_nested mail


In [10]:
# the familiar NameError because mail existed only in namespaces of functions and once they were executed their namespaces were deleted.
print(mail)

NameError: name 'mail' is not defined

So let's have a look at the first code block now.

## Why did do_local() did not affect the output?
So do_local did not change the output of the print statement because of the following:
> To rebind variables found outside of the innermost scope, the nonlocal statement can be used; if not declared nonlocal, those variables are read-only (an attempt to write to such a variable will simply create a new local variable in the innermost scope, leaving the identically named outer variable unchanged).

>A special quirk of Python is that – if no global or nonlocal statement is in effect – assignments to names always go into the innermost scope.

The innermost scope was the scope of do_local() as it is the innermost function at that specific moment and as the variable spam was outside the scope of do_local the assignment did not change spam but instead created a local variable spam inside the do_local() scope with the same name. But after the execution of do_local() that namespace and with it its scope were deleted again.
> The local namespace for a function is created when the function is called, and deleted when the function returns or raises an exception that is not handled within the function.

## Why did do_non_local() affect the output=
Consequently, the use of nonlocal succeeded in assigning the value "nonlocal spam" to the spam variable. As repetitive it might sound:
> To rebind variables found outside of the innermost scope, the nonlocal statement can be used

## Why did do_global() did not affect the output inside the function?
> If a name is declared global, then all references and assignments go directly to the middle scope containing the module’s global names.  

Thus, the search started at the innermost scope, now the scope of scope_test() and here we assigned the value "nonlocal spam" with the nonlocal statement previously. After the execution of the scope_test() its scope is also deleted and as we assigned the value of "global spam" to spam for the middle scope (where the module's global names) the print function will also print this value.
